In [1]:
import numpy as np
from random import gauss
import pandas as pd
from scipy.stats import norm
import yfinance as yf

In [2]:
stock_price = 19265.80 #current market price
strike_price = 19500.00
call = 311.85
put = 378.00
expiry = 41/252 #26th October 2023

#input data for NIFTY 50 19000 as on 25th August 2023

### So to create simulations for asset pricing we need to use stochastic differential equation.

### S(t)=S(o)e(μ−σ^2)dt+σdW(t)
#### Where S(t) is Simulated Price and S(o) is the Spot Price or Market Price

In [3]:
def generate_asset_price(stockPrice,impliedVolatility,riskFreerateofReturn,timeToMaturity):
    return stockPrice*np.exp((riskFreerateofReturn-impliedVolatility**2)*timeToMaturity+impliedVolatility*np.sqrt(timeToMaturity)*gauss(0,1))

In [4]:
def call_payoff(strikePrice,stockPrice):
    return max(stockPrice-strikePrice,0)

def put_payoff(strikePrice,stockPrice):
    return max(strikePrice-stockPrice,0)

In [5]:
StockPrice = stock_price
StrikePrice = strike_price
TimetoMaturity = expiry
RiskFreerate = 0.1
ImpliedVolatility = 0.114 #Volatility Index as on 25th August 2023
#ImpliedVolatility = (yf.Ticker("^INDIAVIX")).history(period="1d")["Close"][0]
dividend = 0
#print(ImpliedVolatility)

In [6]:
numberofsimulations = 10000

Call_payoff = []
Put_payoff = []

#creating new lists in which values will be appended in order to get the list for call and put option prices


discountfactor = np.exp(-RiskFreerate*TimetoMaturity)
#print(discountfactor)

In [7]:
for i in range(numberofsimulations):
    StockPriceAT = round(generate_asset_price(StockPrice,ImpliedVolatility,RiskFreerate,TimetoMaturity),2)
    Call_payoff.append(call_payoff(StrikePrice,StockPriceAT))
    Put_payoff.append(put_payoff(StrikePrice,StockPriceAT))
#print(sum(Call_payoff)/numberofsimulations)
#print(sum(Put_payoff)/numberofsimulations)


In [8]:
call_option_price = discountfactor*(sum(Call_payoff)/numberofsimulations)
#print(call_option_price)
put_option_price = discountfactor*(sum(Put_payoff)/numberofsimulations)
#print(put_option_price)

In [9]:
def black_scholes_call(stock_price,strike_price,TimetoMaturity,RiskFreerate,dividend,ImpliedVolatility):
    
    #Inputs
    #stock_price = Current stock Price
    #strike_price = Strike Price
    #TimetoMaturity = Time to maturity 1 year = 1, 1 months = 1/12
    #RiskFreerate = risk free interest rate
    #dividend = dividend yield
    #ImpliedVolatility = volatility 
    
    #Output
    # call_price = value of the option 
    
    d1 = (np.log(stock_price/strike_price) + (RiskFreerate - dividend + ImpliedVolatility**2/2)*TimetoMaturity) / (ImpliedVolatility*np.sqrt(TimetoMaturity))
    d2 = d1 - ImpliedVolatility* np.sqrt(TimetoMaturity)
    
    call = stock_price * np.exp(-dividend*TimetoMaturity)* norm.cdf(d1) - strike_price * np.exp(-RiskFreerate*TimetoMaturity)*norm.cdf(d2)
    return call

a = black_scholes_call(stock_price,strike_price,TimetoMaturity,RiskFreerate,dividend,ImpliedVolatility)
b = a + strike_price*np.exp(-RiskFreerate*TimetoMaturity) - stock_price #using call-put parity
print(a)
print(b)

394.36070448713144
313.8657599415201


In [10]:
print("Call Option Price from the Monte Carlo Simulation is",round(call_option_price,2))
print("Put Option Price from the Monte Carlo Simulation is",round(put_option_price,2))
print("Call Option Price from the Black Scholes Model is",round(a,2))
print("Call Option Price from market data is",round(call,2))

Call Option Price from the Monte Carlo Simulation is 383.09
Put Option Price from the Monte Carlo Simulation is 313.65
Call Option Price from the Black Scholes Model is 394.36
Call Option Price from market data is 311.85


## Conclusion

In [11]:
if call_option_price < call:
    print("As the call option price is less than the real option price, which means option is underpriced and it is suggested to sell the option")
else:
    print("As the call option price is less than the real option price, which means option is overpriced and it is suggested to hold/buy the option")

As the call option price is less than the real option price, which means option is overpriced and it is suggested to hold/buy the option
